Seismic data is a neat thing. You can imagine it like an ultra-sound of the subsurface. However, in an ultra-sound, we use much smaller wavelengths to image our body. Seismic data usually has wavelengths around 1m to 100m. That has some physical implications, but for now, we don't have to deal with that. It's just something to keep in mind while thinking about resolution. 

Imaging salt has been a huge topic in the seismic industry, basically since they imaged salt the first time. The Society of Exploration geophysicist alone has over 10,000 publications with the [keyword salt](https://library.seg.org/action/doSearch?AllField=salt). Salt bodies are important for the hydrocarbon industry, as they usually form nice oil traps. So there's a clear motivation to delineate salt bodies in the subsurface. If you would like to do a deep dive, you can see [this publication](https://www.iongeo.com/content/documents/Resource%20Center/Articles/INT_Imaging_Salt_tutorial_141101.pdf)

Seismic data interpreters are used to interpreting on 2D or 3D images that have been heavily processed. The standard work of [seismic data analysis](https://wiki.seg.org/wiki/Seismic_Data_Analysis) is open access.
You'll find sections on Salt in there as well (https://wiki.seg.org/wiki/Salt-flank_reflections and https://wiki.seg.org/wiki/Salt_flanks). The seismic itself is pretty "old" in the publication, and you're dealing with data that is less noisy here, which is nice.

[![Seismic Data with salt CC-BY-SA Yilmaz](https://wiki.seg.org/images/1/14/Ch05_fig0-1.png)](https://wiki.seg.org/wiki/Salt-flank_reflections#/media/File:Ch05_fig0-1.png)
Caption: Figure 5.0-1  Conflicting dips associated with salt flanks: (a) CMP stack without dip-moveout correction; (b) time migration of the stack in (a); (c) the stack with dip-moveout correction; (d) time migration of the stack in (c). CC-BY-SA Yilmaz.

Interpretation on seismic images has long used texture attributes, to identify better and highlight areas of interest. These can be seen like feature maps on the texture of the seismic. For salt, you will notice that the texture in the salt masks is rather chaotic, where the surrounding seismic is more "striped". You can think of Earth as layered. Sand gets deposited on top of existing sand. In comes salt, which is behaving very much, unlike other rocks. There is an entire research branch dedicated to salt tectonics, that is the movement of salt in the subsurface. To give you the gist, these salt diapirs form from salt layers somewhere else that were under much pressure. These started to flow (behave ductile) and find a way into other layers above. I have written a bit about salt on [my blog](http://the-geophysicist.com/the-showroom-data-for-my-thesis).

One common seismic attribute is called "chaos" or "seismic disorder". So if you talk to cynic geophysicists, you'll hear "that deep learning better outperform the Chaos attribute". A good starting point is [this publication](http://www.chopraseismic.com/wp-content/uploads/2016/08/Chopra_Marfurt_TLE_Aug2016-LowRes.pdf).

Recently, geoscience has started to adopt deep learning, and it has seen a clear boom, particularly in imaging salt. Code for automatic seismic interpretation can be found here: 

+ https://github.com/waldeland/CNN-for-ASI
+ https://github.com/bolgebrygg/MalenoV
+ https://github.com/crild/facies_net

You will notice that these solutions load a specific SEG-Y file, which luckily we don't have to bother with. TGS provided some nice PNG files instead. However, you can glean some information from them how to approach seismic data. If you find you need some geophysical helpers, you can [import Bruges](https://github.com/agile-geoscience/bruges)

Let's dive in for now.


In [1]:
import os
import sys
import random
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import cv2

from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Lambda
from keras.layers.core import Dropout, Flatten
from keras.layers import BatchNormalization,Concatenate
from keras.layers.convolutional import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

import tensorflow as tf

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

/home/azehady/.local/lib/python2.7/site-packages/pandas/_libs/__init__.py:4: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .tslib import iNaT, NaT, Timestamp, Timedelta, OutOfBoundsDatetime
/home/azehady/.local/lib/python2.7/site-packages/pandas/__init__.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import (hashtable as _hashtable,
/home/azehady/.local/lib/python2.7/site-packages/pandas/core/dtypes/common.py:6: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import algos, lib
/home/azehady/.local/lib/python2.7/site-packages/pandas/core/util/hashing.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import hashing, tslib
/home/azehady/.local/lib/python2.7/site-packages/pandas/core/indexes/base.py:7: Runtime

/home/azehady/.local/lib/python2.7/site-packages/pywt/__init__.py:16: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._extensions._pywt import *
/home/azehady/.local/lib/python2.7/site-packages/pywt/_swt.py:1: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._extensions._swt import swt_max_level, swt as _swt, swt_axis as _swt_axis
/home/azehady/.local/lib/python2.7/site-packages/h5py/__init__.py:36: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._conv import register_converters as _register_converters
/home/azehady/.local/lib/python2.7/site-packages/h5py/__init__.py:45: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import h5a, h5d, h5ds, h5f, h5fd, h5g, h5r, h5s, h5t, h5p, h5z
/home/azehady/.local/lib/python2.7/site-packages/h5py/_hl/group.py:22: Runt

In [2]:
# Set some parameters
im_width = 128
im_height = 128
im_chan = 3
path_train = '../input/train/'
path_test = '../input/test/'

# Data Exploration
Let's look at some data. We can see that TGS chose to use very varied data by inspecting. That is great and adresses a problem in deep learning geoscience at the moment. We build models on one type of seismic and have no idea whether it generalizes.

In [ ]:
ids= ['1f1cc6b3a4','5b7c160d0d','6c40978ddf','7dfdf6eeb8','7e5a6e5013']
plt.figure(figsize=(20,10))
for j, img_name in enumerate(ids):
    q = j+1
    img = load_img('../input/train/images/' + img_name + '.png')
    img_mask = load_img('../input/train/masks/' + img_name + '.png')
    
    plt.subplot(1,2*(1+len(ids)),q*2-1)
    plt.imshow(img)
    plt.subplot(1,2*(1+len(ids)),q*2)
    plt.imshow(img_mask)
plt.show()

We have many examples without salt, as you can see by the masks that are entirely dark. That's great, an algorithm we build will then know that patches exist entirely without salt. Talk about biasing your data.

We can draw heavily on other work, instead of regurgitating the geophysics work that has been done before. I mentioned that seismic is kind of like ultrasound. So I had a look at https://www.kaggle.com/keegil/keras-u-net-starter-lb-0-277

Let's throw a Unet at our data. I am blatanly stealing from Ketil at this point. All credit goes to him and his nice code.
First we'll need to get our data into a shape that works for U-Nets. That means, it should be a power of 2. Let's do it quick and dirty for now, but eventually, consider aliasing and all that fun.

In [ ]:
train_ids = next(os.walk(path_train+"images"))[2]
test_ids = next(os.walk(path_test+"images"))[2]

In [ ]:
# Image Data Augmentation
# https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
# https://machinelearningmastery.com/image-augmentation-deep-learning-keras/
# https://towardsdatascience.com/image-augmentation-for-deep-learning-using-keras-and-histogram-equalization-9329f6ae5085

In [ ]:
# Get and resize train images and masks
R_chan = 0
G_chan = 1
B_chan = 2

X_train_orig_img = [None]
X_train_gray_img = [None]
X_train_rgb_img = np.zeros((len(train_ids), im_height, im_width, 3), dtype=np.uint8)
Y_train_mask_img = np.zeros((len(train_ids), im_height, im_width, 3), dtype=np.uint8)
Y_train_orig_mask_img = [None]

X_train = np.zeros((len(train_ids), im_height, im_width, 1), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), im_height, im_width, 1), dtype=np.bool)

X_train_new = np.zeros((len(train_ids), im_height, im_width, 3), dtype=np.uint8)
Y_train_new = np.zeros((len(train_ids), im_height, im_width, 3), dtype=np.uint8)

img_ids = []
print('Getting and resizing train images and masks ... ')
sys.stdout.flush()
for n, id_ in tqdm_notebook(enumerate(train_ids), total=len(train_ids)):
    path = path_train
    img_ids.append(id_)
    img = load_img(path + '/images/' + id_ )
    
#     r_id = random.randint(0, 100)
#     id_ = train_ids[r_id]
#     print(r_id)
#     img = load_img(path + '/images/' + id_ )

    x_img = img_to_array(img)
    X_train_orig_img.append(x_img)
    x_img = resize(x_img, (im_width, im_height, 3), mode='constant', preserve_range=True)
    X_train_rgb_img[n] = x_img
    x = img_to_array(img)[:,:,1]
    #X_train_gray_img.append(x)
    x = resize(x, (im_width, im_height, 1), mode='constant', preserve_range=True)
    X_train[n] = x
    mask_img = load_img(path + '/masks/' + id_)
    Y_train_orig_mask_img.append(mask_img)
    mask = img_to_array(mask_img)
    mask_img = resize(mask, (im_width, im_height, 3), mode='constant', preserve_range=True)
    Y_train_mask_img[n] = mask_img
    Y_train[n] = resize(mask[:,:,1], (im_width, im_height, 1), mode='constant', preserve_range=True)
    #res = cv2.bitwise_and(img, img, mask = mask)
    
    res1 = mask_img * x_img
    mask_inv = 255.0 - mask_img
    res2 = mask_inv * x_img

    for x1 in range(im_width):
        for x2 in range(im_height):
            if x1 < im_width - 1 and x2 < im_height - 1:
                if  res2[x1,x2, R_chan] == 0 and res2[x1,x2, G_chan] == 0 and res2[x1,x2, B_chan] == 0: # when green channel is 0, color with red
                    res2[x1,x2, G_chan] = 100
                #else:
                    #res2[x1, x2, R_chan] = 100
                

    res3 = res1+res2
    res4 = res3 * mask_img
    res5 = res2 * mask_img
    
    X_train_new[n] = res2
    Y_train_new[n] = res5
    
#     break
    
# plt.subplot(1,5,1)
# plt.imshow(res1)
# plt.subplot(1,5,2)
# plt.imshow(res2)
# plt.subplot(1,5,3)
# plt.imshow(res3)
# plt.subplot(1,5,4)
# plt.imshow(res4)
# plt.subplot(1,5,5)
# plt.imshow(res5)
# plt.show()

# plt.subplot(1,2,1)
# plt.imshow(x_img)
# plt.subplot(1,2,2)
# plt.imshow(mask_img)
# plt.show()

# plt.subplot(1,2,1)
# plt.imshow(res3)
# plt.subplot(1,2,2)
# plt.imshow(res5)
# plt.show()
    
print('Done!')

In [ ]:
# Check if training data looks all right
ix = random.randint(0, len(train_ids))
print(X_train[ix].shape)
#plt.imshow(np.dstack((X_train[ix],X_train[ix],X_train[ix])))
plt.imshow(X_train_new[ix])
plt.show()

print(Y_train[ix].shape)
tmp = np.squeeze(Y_train_new[ix]).astype(np.float32)
#plt.imshow(np.dstack((tmp,tmp,tmp)))
plt.imshow(tmp)
plt.show()

# Train Model
Our task, just like the segmentation task for nuclei, is evaluated on the mean IoU metric. This one isn't in keras, but obviously, we're stealing this one too from Ketil.

In [ ]:
smooth = 1.


def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

# Define IoU metric
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

This is the fun part. Building the sequential Model. The U-Net is basically looking like an Auto-Encoder with shortcuts. 

We're also sprinkling in some earlystopping to prevent overfitting. If you're running this on kaggle, this is the point, you want to have GPU support.

In [ ]:
def conv_block(m, dim, acti, bn, res, do=0):
	n = Conv2D(dim, 3, activation=acti, padding='same')(m)
	n = BatchNormalization()(n) if bn else n
	n = Dropout(do)(n) if do else n
	n = Conv2D(dim, 3, activation=acti, padding='same')(n)
	n = BatchNormalization()(n) if bn else n
	return Concatenate()([m, n]) if res else n

def level_block(m, dim, depth, inc, acti, do, bn, mp, up, res):
	if depth > 0:
		n = conv_block(m, dim, acti, bn, res)
		m = MaxPooling2D()(n) if mp else Conv2D(dim, 3, strides=2, padding='same')(n)
		m = level_block(m, int(inc*dim), depth-1, inc, acti, do, bn, mp, up, res)
		if up:
			m = UpSampling2D()(m)
			m = Conv2D(dim, 2, activation=acti, padding='same')(m)
		else:
			m = Conv2DTranspose(dim, 3, strides=2, activation=acti, padding='same')(m)
		n = Concatenate()([n, m])
		m = conv_block(n, dim, acti, bn, res)
	else:
		m = conv_block(m, dim, acti, bn, res, do) # do dropout for last layer
	return m

def UNet(img_shape, out_ch=1, start_ch=64, depth=4, inc_rate=2., activation='relu', 
		 dropout=0.5, batchnorm=False, maxpool=True, upconv=True, residual=False):
	i = Input(shape=img_shape)
	o = level_block(i, start_ch, depth, inc_rate, activation, dropout, batchnorm, maxpool, upconv, residual)
	o = Conv2D(out_ch, 1, activation='sigmoid')(o)
	return Model(inputs=i, outputs=o)

In [ ]:
# Build U-Net model
#img_shape = (im_height, im_width, im_chan)
img_shape = (im_height, im_width, 3)
inputs = Input(img_shape)
s = Lambda(lambda x: x / 255) (inputs)

model = UNet(img_shape, out_ch = 3, start_ch=8, depth=4, dropout=0.5, batchnorm=True)
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['mean_iou'])
model.compile(optimizer='adam', loss=dice_coef_loss, metrics = [dice_coef])
model.summary()


In [ ]:
print(X_train_new.shape)
print(Y_train_new.shape)

In [ ]:
earlystopper = EarlyStopping(patience=5, verbose=1)
#checkpointer = ModelCheckpoint('model-tgs-salt-3-dropout-0.5-batchnorm-accuracy.h5', verbose=1, save_best_only=True)
checkpointer = ModelCheckpoint('model-tgs-salt-3-dropout-0.5-batchnorm-dice_coef.h5', verbose=1, save_best_only=True)
results = model.fit(X_train_new, Y_train_new, validation_split=0.1, batch_size=8, epochs=100, 
                     callbacks=[earlystopper, checkpointer])

In [ ]:
loss = results.history['loss']
val_loss = results.history['val_loss']

acc = results.history['acc']
val_acc = results.history['val_acc']

epochs = range(len(loss))

plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training vs Validation loss')
plt.legend()

plt.figure()
plt.show()

plt.plot(epochs, acc, 'bo', label='Training Acc')
plt.plot(epochs, val_acc, 'b', label='Validation Acc')
plt.title('Training vs Validation acc')
plt.legend()

plt.figure()
plt.show()

In [ ]:

# val_loss = results.history['val_loss']
# val_mean_iou = results.history['val_mean_iou']
# mean_iou = results.history['mean_iou']

# epochs = range(len(val_loss))

# plt.plot(epochs, mean_iou, 'bo', label='Validation Loss')
# plt.plot(epochs, val_mean_iou, 'b', label='Validation mean IOU')
# plt.title('')
# plt.legend()

# plt.figure()
# plt.show()

In [ ]:
loss = results.history['loss']
val_loss = results.history['val_loss']
dice_coef = results.history['dice_coef']
val_dice_coef = results.history['val_dice_coef']

epochs = range(len(val_loss))

plt.plot(epochs, loss, 'bo', label='Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training vs Validation Loss')
plt.legend()

plt.figure()
plt.show()

plt.plot(epochs, dice_coef, 'bo', label='Dice Coef')
plt.plot(epochs, val_dice_coef, 'b', label='Validation Dice Coef')
plt.title('Training vs Validation Dice Coef')
plt.legend()

plt.figure()
plt.show()

# Test Data
First we'll get the test data. This takes a while, it's 18000 samples.

In [ ]:
# Get and resize test images
#X_test = np.zeros((len(test_ids), im_height, im_width, 1), dtype=np.uint8)
X_test_new = np.zeros((len(test_ids), im_height, im_width, 3), dtype=np.uint8)

sizes_test = []
print('Getting and resizing test images ... ')
sys.stdout.flush()
for n, id_ in tqdm_notebook(enumerate(test_ids), total=len(test_ids)):
    path = path_test
    img = load_img(path + '/images/' + id_)
    x = img_to_array(img)
    x_img = x
    x = x[:,:,1]
    sizes_test.append([x.shape[0], x.shape[1]])
    x = resize(x, (128, 128, 1), mode='constant', preserve_range=True)
    x_new = resize(x_img, (128, 128, 3), mode='constant', preserve_range=True)
    #X_test[n] = x
    X_test_new[n] = x_new

print(X_test_new.shape)
print('Done!')

In [ ]:
# Define IoU metric
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

# Predict on train, val and test
#model = load_model('model-tgs-salt-3-dropout-0.5-batchnorm.h5', custom_objects={'mean_iou': mean_iou})
#model = load_model('model-tgs-salt-3-dropout-0.5-batchnorm-accuracy.h5')
model = load_model('model-tgs-salt-3-dropout-0.5-batchnorm-dice_coef.h5', 
                   custom_objects={'dice_coef': dice_coef, 'dice_coef_loss': dice_coef_loss})

preds_train = model.predict(X_train_new[:int(X_train.shape[0]*0.9)], verbose=1)
preds_val = model.predict(X_train_new[int(X_train.shape[0]*0.9):], verbose=1)
preds_test = model.predict(X_test_new, verbose=1)

# Threshold predictions
preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_val_t = (preds_val > 0.5).astype(np.uint8)
preds_test_t = (preds_test > 0.5).astype(np.uint8)



In [ ]:
np.save('../working/Outputpreds_test', preds_test)

In [ ]:
preds_test_l = np.load('../working/preds_test.npy')

In [ ]:
# del X_train
# del X_train_orig_img
# del X_train_rgb_img
# del X_train_gray_img
# del Y_train

# Create list of upsampled test masks
# preds_test_upsampled = []
# for i in tnrange(len(preds_test)):
#     preds_test_upsampled.append(resize(np.squeeze(preds_test[i]), 
#                                        (sizes_test[i][0], sizes_test[i][1]), 
#                                        mode='constant', preserve_range=True))

In [ ]:
#preds_test_upsampled[0].shape

We'll look at it again, just to be sure.

In [ ]:
# # Perform a sanity check on some random training samples
# ix = random.randint(0, len(preds_train_t))
# #plt.imshow(np.dstack((X_train[ix],X_train[ix],X_train[ix])))
# plt.imshow(X_train_new[ix])
# plt.show()
# # tmp = np.squeeze(Y_train[ix]).astype(np.float32)
# # plt.imshow(np.dstack((tmp,tmp,tmp)))
# plt.imshow(Y_train_new[ix])
# plt.show()

# tmp = np.squeeze(preds_train_t[ix]).astype(np.float32)
# #plt.imshow(np.dstack((tmp,tmp,tmp)))
# plt.imshow(tmp)
# plt.show()

# Prepare Submission
We need to prepare the submission. A nice CSV with predictions. All of this is one to one from Ketil and does not differ from any of the other segmentation tasks. Check them out to improve on this.

In [ ]:
def RLenc(img, order='F', format=True):
    """
    img is binary mask image, shape (r,c)
    order is down-then-right, i.e. Fortran
    format determines if the order needs to be preformatted (according to submission rules) or not

    returns run length as an array or string (if format is True)
    """
    bytes = img.reshape(img.shape[0] * img.shape[1], order=order)
    runs = []  ## list of run lengths
    r = 0  ## the current run length
    pos = 1  ## count starts from 1 per WK
    for c in bytes:
        if (c == 0):
            if r != 0:
                runs.append((pos, r))
                pos += r
                r = 0
            pos += 1
        else:
            r += 1

    # if last run is unsaved (i.e. data ends with 1)
    if r != 0:
        runs.append((pos, r))
        pos += r
        r = 0

    if format:
        z = ''

        for rr in runs:
            z += '{} {} '.format(rr[0], rr[1])
        return z[:-1]
    else:
        return runs

# pred_dict = {fn[:-4]:RLenc(np.round(preds_test_upsampled[i])) for i,fn in tqdm_notebook(enumerate(test_ids))}

In [ ]:
# sub = pd.DataFrame.from_dict(pred_dict,orient='index')
# sub.index.names = ['id']
# sub.columns = ['rle_mask']
# sub.to_csv('submission-dropout-0.5-batchnorm-dice-coef.csv')
#model-tgs-salt-3-dropout-0.5-batchnorm.h5